In [15]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv("/dlabdata1/lugeon/websites_1000_5cat.gz", names=['uid', 'url', 'cat0'])

In [22]:
df.head()

,uid,url,cat0
0,NaN,url,cat0
1,192733.0,http://www.geocities.com/adelaide65mustang/,Arts
2,162532.0,http://www.skyeklad.com/,Arts
3,180561.0,http://www.carliorgani.it/,Arts
4,179143.0,http://www.djaili.com/,Arts


In [24]:
df[df.uid == 60277].url

433    http://origami.ousaan.com/
Name: url, dtype: object

In [11]:
df[df.emb.isnull()]

,Unnamed: 0,emb,len,cat
0,0,NaN,0,Arts
34,64,NaN,0,Arts
181,348,NaN,1,Arts
183,350,NaN,1,Arts
192,370,NaN,1,Arts
...,...,...,...,...
2613,4757,NaN,3,Science
2641,4806,NaN,1,Science
2642,4807,NaN,0,Science
2655,4834,NaN,0,Science


In [18]:
def f():
    return 'a', 'b'

df['a'] = df.apply(lambda x: f(), axis=1)

In [20]:
df['b'] = df.apply(lambda x: x.a[1], axis=1)

In [21]:
df

,Unnamed: 0,emb,cat,"(a, b)",a,b
0,0,NaN,Arts,"(a, b)","(a, b)",b
1,1,"[-0.007258097330729167, -0.041300455729166664,...",Arts,"(a, b)","(a, b)",b
2,2,"[0.069305419921875, -0.0014744808799342106, 0....",Arts,"(a, b)","(a, b)",b
3,3,"[0.06532396589006696, 0.04839869907924107, -0....",Arts,"(a, b)","(a, b)",b
4,4,"[0.0646217248656533, 0.0210318077694286, 0.007...",Arts,"(a, b)","(a, b)",b
...,...,...,...,...,...,...
4115,4995,"[-0.259033203125, -0.064208984375, -0.13989257...",Science,"(a, b)","(a, b)",b
4116,4996,"[0.00269317626953125, 0.026234553410456732, 0....",Science,"(a, b)","(a, b)",b
4117,4997,"[-0.02076325957853861, 0.03759860436584828, 0....",Science,"(a, b)","(a, b)",b
4118,4998,"[-0.03669874323239117, -0.005341988689494583, ...",Science,"(a, b)","(a, b)",b


In [25]:
df = pd.read_json("/dlabdata1/lugeon/websites_1000_5cat_html.json.gz", orient='records', lines=True, nrows=2)

In [26]:
df

,uid,url,html,errcode,cat
0,100943,http://www.360plex.com/news/go/127581/,,200,Arts
1,116334,http://www.allwatchers.com/topics/info_4024.asp,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01//...",404,Arts


In [6]:
df.errcode.value_counts()

200.0    2740
404.0     971
403.0     215
406.0     102
410.0      22
500.0      17
400.0      13
503.0      11
429.0      10
502.0       2
520.0       1
300.0       1
412.0       1
999.0       1
479.0       1
415.0       1
408.0       1
401.0       1
423.0       1
Name: errcode, dtype: int64

In [ ]:
df_json = pd.read_json()